In [1]:
import pandas as pd
import numpy as np
import random as rn
import matplotlib.pyplot as plt
import seaborn as sns
import math, random, json
from pathlib import Path
from datetime import datetime
import statistics
import joblib

import sys
sys.path.append('./rtaUtils')

from rtaUtils import paths, experiment, data_loading

import tensorflow as tf
# from keras.models import Sequential
# from keras.layers import LSTM, Dense, Dropout
# from keras.optimizers import Adam
# from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
# Reproducible
import os
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(42)
rn.seed(1234)
tf.random.set_seed(42)

In [3]:
detailed_metrics = True
num_prev_examples = 1
times = (120,90,60,30,15) # Minutes

# train_data_path = Path('../data/train/train_data_with_val')
# test_data_path  = Path('../data/test/test_data_with_val')
# val_data_path   = Path('../data/val/validation_data')
# models_path     = Path('./models')

# Feature selection
### Features ##################################################################
numeric_feat = [
    'latitude', 'longitude', 'altitude', 'departureDelay', 'vspeed', 'speed', 
    'day_of_week', 'track', 'wind_dir_degrees', 'wind_speed_kt', 
    'visibility_statute_mi', 'max_temp', 'min_temp', 'clouds', 'hav_distance'
]
categoric_feat = [
    'time_of_day', 'operator', 'aerodromeOfDeparture', 'sky_status'
]
objective = ['RTA']
num_features     = len(numeric_feat+categoric_feat)

encoders = joblib.load(paths.utils_path / 'encoder.joblib')

# Model training

In [4]:
train_data = data_loading.load_final_data(month='*', dataset='train', sampling = 60)
# encoders = {}

for feat in categoric_feat:
    le = LabelEncoder().fit(train_data[feat])
    # encoders[feat] = le
    
    train_data[feat] = le.transform(train_data[feat]).reshape(-1,1)

In [ ]:
model = RandomForestRegressor(
    criterion = 'squared_error',
    n_estimators=50,
    n_jobs=2,
    verbose=1)
model.fit(
    train_data[numeric_feat+categoric_feat], 
    train_data[objective].values.reshape((-1,)))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 11.0min


In [ ]:
joblib.dump(model, paths.models_path / 'RF_50units.joblib') 

In [4]:
model = joblib.load(paths.models_path / 'RF_50units.joblib')

In [5]:
val_data   = data_loading.load_final_data(month='*', dataset='val', sampling = 60)
for feat in categoric_feat:
    le = encoders[feat]
    val_data[feat] = le.transform(val_data[feat]).reshape(-1,1)

mae = mean_absolute_error(
        val_data[objective], 
        model.predict(val_data[numeric_feat+categoric_feat]))
print('MAE val set:\t\t{:.5f} segundos'.format(mae))

rmse = math.sqrt(
    mean_squared_error(
        val_data[objective], 
        model.predict(val_data[numeric_feat+categoric_feat])))
print('RMSE val set:\t\t{:.5f} segundos'.format(rmse))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.1s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.1s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


MAE val set:		200.17361 segundos
MAE val set:		200.17361 segundos


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    4.8s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    4.8s


RMSE val set:		353.94125 segundos


[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.2s finished


RMSE val set:		353.94125 segundos


[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.2s finished


In [6]:
data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
total_legs = len(val_data.fpId.unique())
for idx,fpId in enumerate(val_data.fpId.unique()):
    flight = val_data[val_data.fpId == fpId]
    if (idx+1)%25==0:
        print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

    for t in times:
        f = flight[flight.RTA >= t*60]
        f = f.iloc[-num_prev_examples:]

        if len(f)<num_prev_examples: 
            continue
        
        for i in range(num_prev_examples):
            data_at_times[t]['X'].append(f.iloc[i][numeric_feat+categoric_feat])
            data_at_times[t]['Y'].append(f.iloc[i][objective])

print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
for t in times:
    for k,v in data_at_times[t].items():
        data_at_times[t][k] = np.array(data_at_times[t][k])

In [7]:
print('MAE')
for t in times:
    print('{}min: {:.3f}s'.format(t,mean_absolute_error(data_at_times[t]['Y'], 
                                                        model.predict(data_at_times[t]['X']))))

MAE
120min: 275.018s
90min: 240.444s
MAE
120min: 275.018s
90min: 240.444s
60min: 211.393s
60min: 211.393s


C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings

30min: 177.439s
15min: 110.917s


[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished


30min: 177.439s
15min: 110.917s


[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished


In [8]:
test_data   = data_loading.load_final_data(month='*', dataset='test', sampling = 60)
for feat in categoric_feat:
    le = encoders[feat]
    test_data[feat] = le.transform(test_data[feat]).reshape(-1,1)

mae = mean_absolute_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat]))
print('MAE test set:\t\t{:.5f} segundos'.format(mae))

rmse = math.sqrt(
    mean_squared_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat])))
print('RMSE test set:\t\t{:.5f} segundos'.format(rmse))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.2s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.2s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


MAE test set:		190.96800 segundos
MAE test set:		190.96800 segundos


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.0s
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    5.0s


RMSE test set:		327.94489 segundos


[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.4s finished


RMSE test set:		327.94489 segundos


[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    5.4s finished


In [9]:
test_data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
total_legs = len(test_data.fpId.unique())
for idx,fpId in enumerate(test_data.fpId.unique()):
    flight = test_data[test_data.fpId == fpId]
    if (idx+1)%25==0:
        print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

    for t in times:
        f = flight[flight.RTA >= t*60]
        f = f.iloc[-num_prev_examples:]

        if len(f)<num_prev_examples: 
            continue
        
        for i in range(num_prev_examples):
            test_data_at_times[t]['X'].append(f.iloc[i][numeric_feat+categoric_feat])
            test_data_at_times[t]['Y'].append(f.iloc[i][objective])

print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
for t in times:
    for k,v in test_data_at_times[t].items():
        test_data_at_times[t][k] = np.array(test_data_at_times[t][k])

In [10]:
print('MAE')
for t in times:
    print('{}min: {:.3f}s'.format(t,mean_absolute_error(test_data_at_times[t]['Y'], 
                                                        model.predict(test_data_at_times[t]['X']))))


MAE
120min: 274.053s
90min: 221.579s
60min: 195.967s
MAE
120min: 274.053s
90min: 221.579s
60min: 195.967s


C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings

30min: 174.116s
15min: 107.001s


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished


30min: 174.116s
15min: 107.001s


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished


In [5]:
# Future data
test_data   = data_loading.load_final_data(month='202210', dataset='test', sampling = 60)
for feat in categoric_feat:
    le = encoders[feat]
    test_data[feat] = le.transform(test_data[feat]).reshape(-1,1)

test_data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
total_legs = len(test_data.fpId.unique())
for idx,fpId in enumerate(test_data.fpId.unique()):
    flight = test_data[test_data.fpId == fpId]
    if (idx+1)%25==0:
        print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

    for t in times:
        f = flight[flight.RTA >= t*60]
        f = f.iloc[-num_prev_examples:]

        if len(f)<num_prev_examples: 
            continue
        
        for i in range(num_prev_examples):
            test_data_at_times[t]['X'].append(f.iloc[i][numeric_feat+categoric_feat])
            test_data_at_times[t]['Y'].append(f.iloc[i][objective])

print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
for t in times:
    for k,v in test_data_at_times[t].items():
        test_data_at_times[t][k] = np.array(test_data_at_times[t][k])
        
mae = mean_absolute_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat]))
print('MAE test set:\t\t{:.5f} segundos'.format(mae))

rmse = math.sqrt(
    mean_squared_error(
        test_data[objective], 
        model.predict(test_data[numeric_feat+categoric_feat])))
print('RMSE test set:\t\t{:.5f} segundos'.format(rmse))
        
print('MAE')
for t in times:
    print('{}min: {:.3f}s'.format(t,mean_absolute_error(test_data_at_times[t]['Y'], 
                                                        model.predict(test_data_at_times[t]['X']))))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    4.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    4.3s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.


MAE test set:		235.11319 segundos


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    3.3s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    3.6s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished
C:\Users\Jorge\anaconda3\envs\tf\lib\site-packages\sklear

RMSE test set:		426.89709 segundos
MAE
120min: 357.823s
90min: 263.886s
60min: 227.582s
30min: 190.062s
15min: 107.987s


[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done  50 out of  50 | elapsed:    0.0s finished


In [ ]:


# Categorical to numerical conversion
encoders = {}
for feat in categoric_feat:
    le = LabelEncoder().fit(train_data[feat])
    encoders[feat] = le
    
    train_data[feat] = le.transform(train_data[feat]).reshape(-1,1)
    
# Normalization to [0,1] range
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(train_data[numeric_feat+categoric_feat+objective])

scaled_train_data = scaler.transform(train_data[numeric_feat+categoric_feat+objective])

# Conversion to supervised problem
# Data formatting
train_data['idx'] = train_data.index
# Assumes that data are sorted by flight/leg and timestamp.
indices_train = train_data.groupby('leg').agg(first=('idx', 'first'), last=('idx', 'last'))

train_X, train_Y = [],[]
for first, last in indices_train.values:
    temp_x, temp_y = create_dataset(scaled_train_data[first:last,:], lookback)
    train_X.append(temp_x)
    train_Y.append(temp_y)
    
train_X = np.concatenate(train_X, axis=0)
train_Y = np.concatenate(train_Y, axis=0)

In [ ]:

for feat in categoric_feat:
    val_data[feat]   = le.transform(val_data[feat]).reshape(-1,1)
scaled_val_data = scaler.transform(val_data[numeric_feat+categoric_feat+objective])

# Data formatting
val_data['idx'] = val_data.index
indices_val = val_data.groupby('leg').agg(first=('idx', 'first'), last=('idx', 'last'))

val_X, val_Y = [],[]
for first, last in indices_val.values:
    temp_x, temp_y = create_dataset(scaled_val_data[first:last,:], lookback)
    val_X.append(temp_x)
    val_Y.append(temp_y)

val_X = np.concatenate(val_X, axis=0)
val_Y = np.concatenate(val_Y, axis=0)

In [8]:
# Multiple messages
if detailed_metrics:
    data_at_times = {x:{'X':[], 'Y':[]} for x in times}
    
    total_legs = len(val_data.leg.unique())
    for idx,leg in enumerate(val_data.leg.unique()):
        flight = val_data[val_data.leg == leg]
        if (idx+1)%25==0:
            print('{}/{}'.format(idx+1,total_legs).ljust(10,' '), end='\r')

        for t in times:
            f = flight[flight.RTA >= t*60]
            f = f.iloc[-lookback-num_prev_examples:]
            
            if len(f)<lookback+num_prev_examples: 
                continue
            
            scaled_times_data = scaler.transform(f[numeric_feat+categoric_feat+objective])
            test_X, test_Y    = create_dataset(scaled_times_data, lookback)
            for i in range(num_prev_examples):
                data_at_times[t]['X'].append(test_X[i])
                data_at_times[t]['Y'].append(test_Y[i])
            
    print('{}/{} - Done'.format(idx+1,total_legs).ljust(10,' '), end='\r')
    for t in times:
        for k,v in data_at_times[t].items():
            data_at_times[t][k] = np.array(data_at_times[t][k])

In [47]:
for f,i in zip(val_data[numeric_feat+categoric_feat].columns, model.feature_importances_):
    print(f, '\t', i)


latitude 	 0.00029618901046454306
longitude 	 0.005216288208400189
altitude 	 0.0063576640106789055
departureDelay 	 0.0002514266013197549
vspeed 	 9.137399246544936e-05
speed 	 0.0035539660724611577
day_of_week 	 3.245619303266305e-05
track 	 0.00013495466836408365
wind_dir_degrees 	 0.0007883066696654517
wind_speed_kt 	 0.00033715693151600926
visibility_statute_mi 	 0.00011893221375816626
max_temp 	 9.370790532208846e-05
min_temp 	 0.00013742827328527973
clouds 	 1.529394837541134e-05
hav_distance 	 0.9816969655543435
time_of_day 	 8.717761596049648e-05
operator 	 0.0004263373690699054
aerodromeOfDeparture 	 0.00036417242674015597
sky_status 	 2.023347767616562e-07
